#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 1:  Modeling Text + Link Analysis + SEO

### 100 points [6% of your final grade]

### Due: Monday, February 8, 2019 by 11:59pm

*Goals of this homework:* Understand the vector space model (TF-IDF, cosine) + BM25 works in searching. Explore real-world challenges of building a graph (in this case, from Epinions), implement and test the classic HITS algorithm over this graph. Experiment with real-world search engine optimization techniques.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via eCampus (look for the homework 1 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: Modeling Text (50 points)

### TF-IDF


First, you will need to download the review.json file from the Resources tab on Piazza, a collection of about 7,000 Yelp reviews we sampled from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge). You'll see that each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field. You need to load the json file first. We already have done some basic preprocessing on the reviews, so you can just tokenize each review using whitespace.

Here you can treat each review as a document. Given a query, you need to calculate its TF-IDF score in each review.  For this homework, we will define the TF-IDF as follows:

`TF = number of times word occurs in a document`

`IDF = log(total number of documents / number of documents containing the word)`

### A) Ranking with simple sums of TF-IDF scores

To start out with, for a multi-word query, we will rank documents by a simple sum of the TF-IDF scores for the query terms in the document. 

Please calculate this TF-IDF sum score for queries `"best bbq"` and `"kid fun and food"`. You need to report the Top-10 reviews with highest TF-IDF scores for each query. Your output should look like this:

Query "best bbq"

Rank Review_ID score

1 dhskfhjskfhs 0.55555

...



Query "kid fun and food"

Rank Review_ID score

1 dhskfhjskfhs 0.55555

...

In [3]:
import math
import json

reviewList = []
with open('/Users/jiangbohuai/Desktop/CSCE670/review.json', 'r') as f:
    for line in f:
        reviewList.append(json.loads(line))

# print(reviewList[0]['review'])
lengthOfReviewList = len(reviewList)
wordSet = set()

for review in reviewList :
    for word in review['review'].split() :
        wordSet.add(word)
        
IDF_Scores = {}

for word in wordSet : 
    IDF_Scores[word] = 0
    
# find the IDF of every word in the reviewList
def computeIDF() :
    for review in reviewList :
        reviewListByWord = review['review'].split()
        reviewSetByWord = set(reviewListByWord)
        for word in reviewSetByWord :
            IDF_Scores[word] = IDF_Scores[word] + 1
    for word in wordSet :
        IDF_Scores[word] = math.log(6751 / IDF_Scores[word])
    return IDF_Scores
idfResult = computeIDF()
print(idfResult['shells'])
# print(computeIDF())
# # result of best bbq in computeIDF[1.9599318584964802, 4.386629122198557]   

6.871535771986557


In [4]:
# return the tf of every review in the reviewList
def computeTF() :
    TF_Scores = []  
    i = 0
    for review in reviewList :
        reviewListByWord = review['review'].split()
        reviewSetByword = set(reviewListByWord)
        reviewListWithoutDuplication = list(reviewSetByword)
        for word in reviewListWithoutDuplication :
            num = reviewListByWord.count(word)
            if i >= len(TF_Scores) :
                TF_Scores.append({word : num})
            else :
                TF_Scores[i][word] = num
        TF_Scores[i]['id'] = review['id']
        i += 1
    return TF_Scores
tfResult = computeTF()
# print(computeTF())



# for tfDict in tfResult :
#     if 'best' in tfDict :
#         print(tfDict['id'], tfDict['best'])

def computeFT_IDF(query) :
    res = []
    for tfDict in tfResult :
        num = 0
        for word in query.split() :
            if word in tfDict :
                num += tfDict[word] * idfResult[word]
        res.append([num, tfDict['id']])    
    res.sort(key=lambda x: x[0])
    res.reverse()
    return res[:10]
# print(computeFT_IDF('best bbq'))
# # print(computeFT_IDF('kid fun and food'))

In [5]:
# Show us the result for "best bbq"
print('rank    ' + 'Review_ID                    ' + 'score')
# print(computeFT_IDF('best bbq'))
res = computeFT_IDF('best bbq')
i = 1
for review in res :
    print(i,'     ', review[1], '     ', review[0])
    i = i + 1

rank    Review_ID                    score
1       YbQvHNrjZJ38mnh5rLuq7w       26.319774733191345
2       P31kXP4oan6ZQm69TN6tIA       21.933145610992785
3       x5esEK6J9XkA_vbvVbG8Gg       19.506448347290707
4       NCfX4AxDvQ3QRyXKtmhVwQ       17.54651648879423
5       mWs26TrBM7ogwCM9UfVJFg       17.54651648879423
6       4WTG1-9mw8YHEyaTu8dQww       15.119819225092153
7       e5INq6DAZn2zMHicKQl07Q       15.119819225092153
8       x3n_l3GhBx78y6jWX4fStg       13.719523009475362
9       8p-KEtrrTmLv-o1mKpUy1A       13.159887366595672
10       1tJ_iJX_KZ3zM_9_GRaGTg       13.159887366595672


In [6]:
# Show us the result for "kid fun and food"
print('rank    ' + 'Review_ID                    ' + 'score')
res = computeFT_IDF('kid fun and food')
i = 1
for review in res :
    print(i,'     ', review[1], '     ', review[0])
    i = i + 1

rank    Review_ID                    score
1       7o_hciiXEMNQkXfVl0F0XQ       22.201231889774093
2       JKLUXUovJCU6kbcdin74NQ       20.014087913721344
3       IA8TOfGKI-Il-70BsB6HgA       18.725471979355085
4       Kytq1NbFIDDCXUculSqT8g       16.78875808815464
5       MF6rPRx9jz-g8S5P_ZIdyg       16.302406482703674
6       bjoedmJ4_DZP5JnfXVaC-w       15.716259660853684
7       I00B-QG5uTKvwCK7x9ejeA       15.662550111334058
8       BVGRJgDJGEhSfgIPCan7vQ       15.477061200988935
9       wMB3cI3-xhxM_BpmppY9RQ       14.794561503546632
10       vTGDEQGp6EPlwdMJUnTb7A       13.910994327397505


### B) Ranking with TF-IDF + Cosine

Instead of using the sum of TF-IDF scores, let's try the classic cosine approach for ranking. You should still use the TF-IDF scores to weigh each term, but now use the cosine between the query vector and the document vector to assign a similarity score. You can try the same two queries as before and report the results. (Top-10 reviews which are similar to the query)

In [7]:
import math
import json
def TfIdfVector() :
    res = []
    for tfDict in tfResult :
        tempDict = {}
        for word in tfDict :
#             if to get rid of 'id'
            if word != 'id' :
                num = idfResult[word] * tfDict[word]
                tempDict[word] = num
        tempDict['id'] = tfDict['id']
        res.append(tempDict)
    return res
TfIdfVector = TfIdfVector()
# print(TfIdfVector)


def squareRoot(targetDict) :
    res = 0
    for word in targetDict :
        if word != 'id' :
            res += targetDict[word] ** 2
    res = math.sqrt(res)
    return res

def vectorMultiplication(query) :
    res = []
    for TfIdfDict in TfIdfVector :
        a = 0
        b = squareRoot(TfIdfDict)
        c = len(query.split())
        for word in query.split() :
            if word in TfIdfDict :
                a = a + TfIdfDict[word] 
        c = math.sqrt(c)
#         if b == 0 :
#             res.append([0, vector[-1]])
#         if b != 0 :
        res.append([(a / b / c), TfIdfDict['id']])
    res.sort(key = lambda x: x[0])
    res.reverse()
    return res[:10]
print(vectorMultiplication('best bbq'))

[[0.5345284475881048, '6wRJtHhvQsS6vOse466_3w'], [0.4360447415638786, 'x5esEK6J9XkA_vbvVbG8Gg'], [0.42255218023429036, 'eAXFFP3GxUfGjQlAZDB_CQ'], [0.40004536886838066, '7LaBODDEaUNRpLPDG_bKtQ'], [0.3577182765572638, 'P31kXP4oan6ZQm69TN6tIA'], [0.3545945072162078, 'ZAn6zB6VOCsJ1OsGRv-NVA'], [0.3399739836785112, '8p-KEtrrTmLv-o1mKpUy1A'], [0.31605256633839557, 'RHWT1KVeIw2FT7i5BP_TVw'], [0.308230511122904, '_fNfowXaxXcYChKukMrYeg'], [0.2990933593101938, 'AEiNkWY-4ggToDeMTd8l1w']]


In [8]:
# Show us the result for "best bbq"
print('rank    ' + 'Review_ID                    ' + 'score')
res = vectorMultiplication('best bbq')
i = 1
for review in res :
    print(i,'     ', review[1], '     ', review[0])
    i = i + 1
    

rank    Review_ID                    score
1       6wRJtHhvQsS6vOse466_3w       0.5345284475881048
2       x5esEK6J9XkA_vbvVbG8Gg       0.4360447415638786
3       eAXFFP3GxUfGjQlAZDB_CQ       0.42255218023429036
4       7LaBODDEaUNRpLPDG_bKtQ       0.40004536886838066
5       P31kXP4oan6ZQm69TN6tIA       0.3577182765572638
6       ZAn6zB6VOCsJ1OsGRv-NVA       0.3545945072162078
7       8p-KEtrrTmLv-o1mKpUy1A       0.3399739836785112
8       RHWT1KVeIw2FT7i5BP_TVw       0.31605256633839557
9       _fNfowXaxXcYChKukMrYeg       0.308230511122904
10       AEiNkWY-4ggToDeMTd8l1w       0.2990933593101938


In [9]:
# Show us the result for "kid fun and food"
print('rank    ' + 'Review_ID                    ' + 'score')
res = vectorMultiplication('kid fun and food')
i = 1
for review in res :
    print(i,'     ', review[1], '     ', review[0])
    i = i + 1


rank    Review_ID                    score
1       IUME6cWFSwH1mSh_1_U81g       0.38678092826505023
2       ag1fnnEmc2yernTW2ur2eg       0.3184083256114468
3       OExraycGW4VxL0Xth1xZ4w       0.28233280921832343
4       37RfMeDMo8QEVAF8yT31Ww       0.20412875547557477
5       x72i0s6q84ouimza6Y3HSQ       0.19734521151849208
6       6xdziQ46TZWKBpKQPNCSGw       0.18596593471006725
7       Pp0h1AxxHpTU-ylBt2mldQ       0.17287031170859368
8       1HshwJX7afs-CKdczFbI5g       0.172033574387773
9       a2xfP0RpJAhioxUUHCX6QQ       0.17094432570094087
10       _AXfMxnvGx6a4L_ZgPCMKA       0.1677263887030493


### C) Ranking with BM25

Finally, let's try the BM25 approach for ranking. Refer to [https://en.wikipedia.org/wiki/Okapi_BM25](https://en.wikipedia.org/wiki/Okapi_BM25) for the specific formula. You should choose k_1 = 1.2 and b = 0.75. You need to report the Top-10 reviews with highest BM25 scores for each query.


In [22]:
# Your code here
# print(len(reviewList))
def avgdl() :
    length = 0
    for review in reviewList :
        length += len(review['review'].split())
    return length / len(reviewList)
avg = avgdl()
# print(avg)

127.88268404680788


In [33]:
def BM25(query) :
    resBM25 = []
    i = 0
    for review in reviewList :
        score = 0
        for word in query.split() :
            if word in tfResult[i] :
                num = tfResult[i][word]
                temp = idfResult[word] * num * 2.2
                temp = temp / (num + 1.2 * (0.25 + 0.25 * len(review['review'].split()) / avg))
                score += temp
        resBM25.append([score, review['id']])
        i += 1
    return resBM25
def printBM25(query) :
    res = BM25(query)
    res.sort(key = lambda x: x[0])
    res.reverse()
    return res[:10]
print(printBM25('best bbq'))

[[11.253601127552948, 'x5esEK6J9XkA_vbvVbG8Gg'], [10.79145311681653, '4WTG1-9mw8YHEyaTu8dQww'], [10.60017213929137, 'e5INq6DAZn2zMHicKQl07Q'], [10.25900208888154, 'xpm6TgDiHaQdEDlErFsqvQ'], [9.768354886994567, '-RApX_RMzJLnpommDpQfKQ'], [9.628351200700767, 'GASAd_gPBY_eWIL9XJwuNA'], [9.59371842711005, 'JB0ryh232GE1UqlFgEIOpg'], [9.060686222171345, '6E7RhJa4kx0ci0-nfv7Ivw'], [8.776483614414637, 'cUPlcoMQdyLxMQQbXYzNkw'], [8.656054774056912, 'P31kXP4oan6ZQm69TN6tIA']]


In [34]:
# Show us the result for "best bbq"
print('rank    ' + 'Review_ID                    ' + 'score')
res = printBM25('best bbq')
i = 1
for review in res :
    print(i,'     ', review[1], '     ', review[0])
    i = i + 1


rank    Review_ID                    score
1       x5esEK6J9XkA_vbvVbG8Gg       11.253601127552948
2       4WTG1-9mw8YHEyaTu8dQww       10.79145311681653
3       e5INq6DAZn2zMHicKQl07Q       10.60017213929137
4       xpm6TgDiHaQdEDlErFsqvQ       10.25900208888154
5       -RApX_RMzJLnpommDpQfKQ       9.768354886994567
6       GASAd_gPBY_eWIL9XJwuNA       9.628351200700767
7       JB0ryh232GE1UqlFgEIOpg       9.59371842711005
8       6E7RhJa4kx0ci0-nfv7Ivw       9.060686222171345
9       cUPlcoMQdyLxMQQbXYzNkw       8.776483614414637
10       P31kXP4oan6ZQm69TN6tIA       8.656054774056912


In [35]:
# Show us the result for "kid fun and food"
print('rank    ' + 'Review_ID                    ' + 'score')
res = printBM25('kid fun and food')
i = 1
for review in res :
    print(i,'     ', review[1], '     ', review[0])
    i = i + 1


rank    Review_ID                    score
1       TVq6HhhJizKM1mReF9hvJQ       10.747579997870728
2       kDwMMrSiB_AlV0erhVigFg       10.545688836713447
3       GKgnNAElM0ybDix0M5xq5w       10.377172640503517
4       vTGDEQGp6EPlwdMJUnTb7A       10.082608145873639
5       UMqvuRtTxJFuWbgT6qO9cg       9.947665595795232
6       O2n7CfnlIRQhKS49dIi-CA       9.887542733516144
7       BLQYsPFFAezpbbF-1dzD4Q       9.817504171484686
8       VuF0YIl7k8qNbDjlg8iAyg       9.661225535543103
9       _HyK3isyGiChhLPaCWn0nQ       9.644089163772726
10       7ptI3mbzmfmDSOcOu0GGrw       9.598327746855187


Briefly discuss the differences you see between the three methods. Is there one you prefer? 

In [ ]:
# I prefer to use BM25;
# Difference : The TF_IDF method is just first calculate the TF and IDF numbers of those words that query has, then multiply them; 
#     but The TF_IDF_COSINE method add one other features, which is taking consideration of the normalization. Through the 
#     normalization, the TF and IDF numbers of those words even the query does not can also influence the score.document with 
#     different length has different score;
#     And the BM25 is even better because it considers some parameters like k1 and b; If we choose different k1 or b, the score can 
#     change.

*add your discussion here*

# Part 2: Link Analysis (40 points)

## A Trust Graph


In this part, we're going to adapt the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find the most trustworthy users. [Epinions.com](https://snap.stanford.edu/data/soc-Epinions1.html) is a general consumer review site with a who-trust-whom online social network. Members of the site can decide whether to ''trust'' each other. All the trust relationships interact and form the Web of Trust which is then combined with review ratings to determine which reviews are shown to the user. (Refer to: Richardson, Matthew, Rakesh Agrawal, and Pedro Domingos. "Trust management for the semantic web." International semantic Web conference. Springer, Berlin, Heidelberg, 2003.)

So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Epinions users and their "trust" on other users (so user = node, trust behavior = edge). Over this Epinions-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

You need to download the *Epinions_trust.txt* file from the Resources tab on Piazza. Each line represents the trust relationship between two users. Here is a toy example. Suppose you are given the following four lines:

* diane trust bob
* charlie trust bob 
* charlie trust alice
* bob trust charlie

The "trust" between each user pair denotes a directed edge between two nodes. E.g., the "diane" node has a directed edge to the "bob" node (as indicated by the first line). 

You should build a graph by parsing the data in the file we provide called *Epinions_trust.txt*. (Note: The edges are binary and directed.)

**Notes:**

* The edges are binary and directed.
* User can't trust himself/herself.
* Later you will need to implement the HITS algorithm on the graph you build here.

In [135]:
with open('/Users/jiangbohuai/Desktop/CSCE670/Epinions_trust.txt', 'r') as f:
        outFile = f.read()
trustList = outFile.split('\n')
# print(trustList)

dic = dict()
for edge in trustList :
    word = edge.split()
    if len(word) != 0 :
        start = word[0]
        end = word[2]
        if start != end :
            if start in dic :
                if end not in dic[start] :
                    dic[start].append(end)
            else :
                dic[start] = [end]
# print(dic)

# def testDuplicate(dic) :
#     for key in dic :
#         if (len(dic[key]) != len(set(dic[key]))) :
#             return 'true'
#     return 'false'
# print(testDuplicate(dic))

def sizeOfGraph(dic) :
    edge = 0
    nodeList = []
    for key in dic :
        edge += len((dic[key]))
        nodeList.append(key)
    for key in dic :
        for value in dic[key] :
            if value not in nodeList : 
                nodeList.append(value)
    return len(nodeList), edge
# print(sizeOfGraph(dic))
# print(len(dic['thechuunt']))

Please show us the size of the graph, i.e., the number of nodes and edges


In [136]:
# Call your function to print out the size of the graph
# How you maintain the graph is totally up to you

print('the number of nodes and the number of edges are shown belown respectively')
print(sizeOfGraph(dic))

the number of nodes and the number of edges are shown belown respectively
(658, 6378)


## HITS Implementation

Your program will return the top 10 users with highest hub and authority scores. The **output** should be like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal scores.
* It is up to you to decide when to terminate the HITS calculation.
* There are HITS implementations out there on the web. However, remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and HITS calculations using a handful of trust relationships, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

In [139]:
# HITS IMPLEMENTATION
# dic = {'A': ['B'], 'B': ['C','D'],'C' : ['D'], 'D' : ['B'], 'E' : ['B']}

import math
def NodeListOfGraph(dic) :
    nodeList = []
    for key in dic :
        nodeList.append(key)
    for key in dic :
        for value in dic[key] :
            if value not in nodeList : 
                nodeList.append(value)
    return nodeList
nodeList = NodeListOfGraph(dic)


def normalization(dictionary) :
    res = 0
    for key in dictionary :
        res += dictionary[key] ** 2
    res = math.sqrt(res)
    return res
    
# get the dictionary to calculate the authority value
dicAuthority = {}
for key in dic :
    for value in dic[key] :
        if value in dicAuthority :
            dicAuthority[value].append(key)
        else :
            dicAuthority[value] = [key]
# print(dic)
# print(dicAuthority)

# .................................initialization
authority = {}
hub = {}
for node in nodeList :
    authority[node] = 1
    hub[node] = 1

# ..................................iteration
i = 0
while i < 10 :
    for key in hub :
        if key in dic :
            hub[key] = 0
            for edge in dic[key] :
                hub[key] += authority[edge]
        else :
            hub[key] = 0
#     print('hub' , hub)
#     print('\n')
    for key in authority :
        if key in dicAuthority :
            authority[key] = 0
            for edge in dicAuthority[key] :
                authority[key] += hub[edge]
        else :
            authority[key] = 0
#     print('authority', authority)
#     print('\n')
#  .................normaliztion

#   the next line is important because if not do so, the normalization(hub) is actually changing when we change hub[key]
    divisor = normalization(hub)
    for key in hub :
        hub[key] = hub[key] / divisor
    for key in authority :
        authority[key] = authority[key] / divisor
    i += 1
#     print('hub', hub)
#     print('\n')
#     print('authority', authority)
#     print('\n')

hubSorted = sorted(hub.items(), key=lambda x: x[1])
hubSorted.reverse()
# print(hubSorted)
authSorted = sorted(authority.items(), key=lambda x: x[1])
authSorted.reverse()
# print(authSorted)

In [140]:
i = 0
j = 0
print('hub scores\n')
while i < 10 :
    print(hubSorted[i][0], hubSorted[i][1])
    i += 1
print('\n')
print('authority scores\n')
while j < 10 :
    print(authSorted[j][0], authSorted[j][1])
    j += 1

hub scores

charles 0.18163967198194106
teanna3 0.17735805756000603
JediKermit 0.1631313056695043
KCFemme 0.15102471058706424
melissasrn 0.14454261197340737
missi31 0.14338346958671092
jeanniekerns 0.1427769407466428
jag2112 0.14200895811996167
mrssmoopy 0.14199810142435626
briandalsmom 0.1392722414414174


authority scores

melissasrn 9.11961779177607
shantel575 7.258262526547998
surferdude7 7.176099304051569
sblaydes 5.814875981525541
tiffer0220 5.740998731065901
opinionated3 5.6395151721850905
patch3boys 4.845509482763947
merlot 4.774319180746799
pogomom 4.665213888045435
chrisceb 4.428135840246835


For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **sajfd hfafbjhd** --- two terms, lower case, no quote. As of today (Jan 24, 2019), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you.
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 5 points for a well-reasoned discussion of your strategy

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?
http://people.tamu.edu/~bohuaijiang.1021/

What's your strategy? (2-4 paragraphs)

First, I make a personal website on people.tamu.edu; Then I mention the sajfd hfafbjhd.
Second, a youtube link is given in this page considering youtube has high authority score.
Then lastly, I put another student's website link which has high rank of the term on google. So I think I can take advantage of that

## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

In [ ]:
# https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary  str to dict
# https://www.quora.com/How-is-the-HITS-algorithm-implemented
# https://www.w3schools.com/python/python_dictionaries.asp
# https://stackoverflow.com/questions/17555218/python-how-to-sort-a-list-of-lists-by-the-fourth-element-in-each-list